<img src="../images/GeoCAT_logo.png" width=400 alt="GeoCAT Logo"></img>

# Plotting with GeoCAT-viz

---

## Learning Objectives
- Know what GeoCAT-viz is and how it can be useful to you

## Prerequisites


| Concepts | Importance |
| --- | --- |
| Basic familiarity with NumPy | Necessary |
| Basic familiarity with Matplotlib | Necessary |
| Basic familiarity with Cartopy | Helpful |
| Basic familiarity with Xarray | Helpful |

- **Time to learn**: *15-20 minutes*


---

<div class="admonition alert alert-danger">
   
This notebook demonstrates geocat-viz functionality not officially released as of the time of this tutorial. 

</div>

## Imports

In [2]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [3]:
# geocat specific imports
from geocat.viz.contourf import Contour
from geocat.viz.taylor import TaylorDiagram
from geocat.viz import cmaps as gvcmaps
from geocat.viz import util as gvutil
import geocat.datafiles as gdf

## Example 1: Contours

### Get the data we want to use

Using geocat-datafiles again, pull the data we're going to use for both versions of the plot

In [4]:
# Open a netCDF data file using xarray default engine and load the data into xarray
ds = xr.open_dataset(gdf.get("netcdf_files/uv300.nc")).isel(time=1)

### Recreating an NCL Contour Plot with current GeoCAT-viz functionality

The first goal of geocat-viz created reusable utility functions that aim to make some of the NCL formatting easier. 

Below is an example from the GeoCAT-examples repository, [NCL_color_1](https://geocat-examples.readthedocs.io/en/latest/gallery/Contours/NCL_color_1.html#sphx-glr-gallery-contours-ncl-color-1-py) using the data we pulled in above and that uses some of the geocat-viz utility functions

<div class="admonition alert alert-success">
   
This section represents the currently released functionality of geocat-viz

</div>

In [ ]:
# Generate figure and set its size in (width, height)
fig = plt.figure(figsize=(10, 8))

# Generate axes using Cartopy to draw coastlines
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(linewidth=0.5, alpha=0.6)

# Use geocat.viz.util convenience function to set axes limits & tick values
gvutil.set_axes_limits_and_ticks(ax,
                                 xlim=(-180, 180),
                                 ylim=(-90, 90),
                                 xticks=np.linspace(-180, 180, 13),
                                 yticks=np.linspace(-90, 90, 7))

# Use geocat.viz.util convenience function to add minor and major tick lines
gvutil.add_major_minor_ticks(ax, labelsize=10)

# Use geocat.viz.util convenience function to make latitude, longitude tick labels
gvutil.add_lat_lon_ticklabels(ax)

# Import the default color map
newcmp = gvcmaps.ncl_default

# Define contour levels
levels = np.arange(-16, 48, 4)

# Define dictionary for kwargs
kwargs = dict(
    levels=levels,
    xticks=np.arange(-180, 181, 30),  # nice x ticks
    yticks=np.arange(-90, 91, 30),  # nice y ticks
    add_colorbar=False,  # allow for colorbar specification later
    transform=ccrs.PlateCarree(),  # ds projection
)

# Contouf-plot U data (for filled contours)
fillplot = ds.U.plot.contourf(ax=ax, cmap=newcmp, **kwargs)

# Create horizontal color bar
# By changing the kwarg `pad`, the colorbar can be moved closer to or farther away from
# the axis parallel to it.
# `pad` defaults to 0.15 for horizontal colorbars
fig.colorbar(fillplot,
             orientation="horizontal",
             ticks=np.arange(-12, 44, 4),
             label='',
             shrink=0.75,
             pad=0.11)

# Plot line contours
ds.U.plot.contour(ax=ax,
                  colors='black',
                  alpha=0.8,
                  linewidths=0.4,
                  linestyles='solid',
                  add_labels=False,
                  levels=levels,
                  transform=ccrs.PlateCarree())

# Use geocat.viz.util convenience function to add titles to left and right of the plot axis.
gvutil.set_titles_and_labels(ax,
                             maintitle="Default Color",
                             lefttitle=ds.U.long_name,
                             lefttitlefontsize=16,
                             righttitle=ds.U.units,
                             righttitlefontsize=16,
                             xlabel="",
                             ylabel="")

# Show the plot
plt.show()

### Recreating the same plot with upcoming GeoCAT-viz functionality

From our previous matplotlib tutorial, you might suspect that this is more lines than technically needed to create a plot that accurately displays the information we want. 

The secondary goal of geocat-viz is to create plotting functions based on matplotlib that can make an estimation of NCL style plots. These functions may not exactly recreate the NCL example scripts, but provide a user-friendly way to make NCL style plots.

<div class="admonition alert alert-danger">
   
This is unreleased functionality not yet avaialble in geocat-viz

</div>

In [ ]:
projection = ccrs.PlateCarree()
levels = np.arange(-16, 48, 4)

Contour(
    ds.U,
    flevels=levels,
    clevels=levels,
    cmap=gvcmaps.ncl_default,
    xlim=(-180, 180),
    ylim=(-90, 90),
    xticks=np.linspace(-180, 180, 13),
    yticks=np.linspace(-90, 90, 7),
    projection=projection,
    maintitle="Default Color",
    lefttitle=ds.U.long_name,
    righttitle=ds.U.units,
)

plt.show()

This wrapper function creates a replication of the original NCL plot with minimal knowledge of geocat-viz helper functions. 

Future planned improvements include the automation of several of the kwargs, letting the internal program take a "guess" at what NCL *would* do.

## Example 2: Taylor Diagrams

<div class="admonition alert alert-warning">
   
This functionality will be officially released in the September GeoCAT release.

</div>

The GeoCAT development of Taylor diagrams started with an [example request issue](https://github.com/NCAR/GeoCAT-examples/issues/323) created on geocat-examples.

As we looked in to making an example of this type of plot, we discovered that there wasn't a good way to create this type of plot using vanilla matplotlib.

### A Taylor Diagram Primer

Taylor diagrams are a "a concise statistical summary of how well patterns match each other in terms of their correlation, their root-mean-square difference and the ratio of their variances" according to the [original AGU paper by Taylor](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2000JD900719) in 2001.



<img src="../images/taylor_paper.png" width=2000 alt="screenshot of title and abstract of Taylor Diagram AGU paper"></img>

Taylor diagrams plot the weighted centered cappern correlations, the ratios of the normalized root-mean-squared differences between the test and reference data sets, and optionally a bias statistic. Below is an example from [NCL's Taylor diagram documentation](https://www.ncl.ucar.edu/Applications/taylor.shtml).

<img src="../images/taylor_3.png" width=600 alt="NCL example taylor diagram"></img>

### Recreating Taylor Diagrams

We're going to demonstrate geocat-viz's `TaylorDiagram` by recreating the above example from the NCL documentation.

In [ ]:
# Case A
CA_ratio = [1.230, 0.988, 1.092, 1.172, 1.064, 0.966, 1.079, 0.781]  # case variance/ref variance
CA_cc = [0.958, 0.973, 0.740, 0.743, 0.922, 0.982, 0.952, 0.433]     # cross corr coef of case to ref

# Case B
CB_ratio = [1.129, 0.996, 1.016, 1.134, 1.023, 0.962, 1.048, 0.852]  # case variance/ref variance
CB_cc = [0.963, 0.975, 0.801, 0.814, 0.946, 0.984, 0.968, 0.647]     # cross corr coef of case to ref

# Create TaylorDiagram instance
tp = TaylorDiagram()

# Add models to Taylor diagram
tp.add_model_set(CA_ratio,
                  CA_cc,
                  color='red',
                  marker='o',
                  label='Case A')

tp.add_model_set(CB_ratio,
                  CB_cc,
                  color='blue',
                  marker='o',
                  label='Case B')

# Add model names
namearr = ['SLP', 'Tsfc', 'Prc', 'Prc 30S-30N', 'LW', 'SW', 'U300', 'Guess']
tp.add_model_name(namearr)

# Add figure legend
tp.add_legend()

# Display the plot
plt.show();


### Comparison